In [1]:
### Fixing import errors of the

import sys
import os

# This code navigates up one directory from the notebook's location ('examples/')
# to get the project's root directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# We check if the path is already in the system path.
# If not, we add it to the beginning of the list.
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added project root to Python path: {project_root}")
else:
    print(f"Project root is already in Python path: {project_root}")

# Optional: You can print the first few paths to verify
print("\nVerifying sys.path:")
for i, path in enumerate(sys.path[:5]):
    print(f"{i}: {path}")

Added project root to Python path: /home/nick/projects/Llama-Index-GliREL-GraphRAG

Verifying sys.path:
0: /home/nick/projects/Llama-Index-GliREL-GraphRAG
1: /usr/lib/python312.zip
2: /usr/lib/python3.12
3: /usr/lib/python3.12/lib-dynload
4: 


In [2]:
from src.GlirelPathExtractor import GlirelPathExtractor 
from llama_index.core import SimpleDirectoryReader, PropertyGraphIndex,Settings

/home/nick/envs/grag-llama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

In [4]:
llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)

Settings.llm = llm
Settings.chunk_size=512
Settings.chunk_overlap=64

In [5]:

embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed2:latest",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embed_model

In [ ]:
documents = SimpleDirectoryReader(
    input_files=["../.data/Demo-Barack_Obama.txt"]
).load_data()

In [7]:
extractor = GlirelPathExtractor(device="cuda")


In [8]:
index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[extractor],
    use_async = False
    
)


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 88862.37it/s]
/home/nick/envs/grag-llama/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
query_engine = index.as_query_engine(
    llm=llm,
    include_text="True",
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
    )

In [10]:
response= query_engine.query("When was obama born?")
response

Response(response='Barack Obama was born on August 4, 1961.', source_nodes=[NodeWithScore(node=TextNode(id_='2ebbc343-f532-46f4-80d6-56e5f73f954b', embedding=None, metadata={'file_path': '../.data/Demo-Barack_Obama.txt', 'file_name': 'Demo-Barack_Obama.txt', 'file_type': 'text/plain', 'file_size': 82744, 'creation_date': '2025-06-26', 'last_modified_date': '2025-06-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8b2f6f9d-014b-46b8-9caf-b4b177ebfaee', node_type='4', metadata={'file_path': '../.data/Demo-Barack_Obama.txt', 'file_name': 'Demo-Barack_Obama.txt', 'file_type': 'text/plain', 'file_size': 82744, 'creation_date': '2025-06-26', 'last_modified_date': '2025-06-26'}, hash='4bef1893d3be2558872a1d4d